# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
gmaps.__version__ # => 0.4.2.rc3
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Loading the output csv file generated in weatherpy to a dataframe
vacation_df=pd.read_csv("../WeatherPy/Output/city_weather.csv")
vacation_df

,Unnamed: 0,City,Country,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Date
0,0,Coyhaique,CL,-45.5752,-72.0662,38.34,99,100,3.67,1626230190
1,1,Punta Arenas,CL,-53.1500,-70.9167,26.71,93,75,3.44,1626229916
2,2,Busselton,AU,-33.6500,115.3333,57.04,67,49,18.25,1626230190
3,3,Ngerengere,TZ,-6.7500,38.1167,67.75,95,51,1.66,1626230191
4,4,Baft,IR,29.2340,56.6057,77.63,33,11,2.66,1626230191
...,...,...,...,...,...,...,...,...,...,...
563,563,Zhigansk,RU,66.7697,123.3711,68.59,65,32,16.73,1626230336
564,564,Woodward,US,36.4337,-99.3904,83.43,54,1,11.50,1626230336
565,565,Honiara,SB,-9.4333,159.9500,87.33,66,92,11.27,1626230052
566,566,Bayir,TR,37.2687,28.2167,64.92,44,0,4.23,1626230337


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configuring gmaps with api key
gmaps.configure(api_key=g_key)

In [4]:
# locating gmap by latitude and longitude and Humidity as weight
locations = vacation_df[['Latitude', 'Longitude']].astype(float)
weights = vacation_df['Humidity'].astype(float)

# plotting base gmap 
fig = gmaps.figure(center=(18.0, 10.0), zoom_level=2)

# Adding heatmap layer over the base map
heat_layer=fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Finding whether any null value exists
vacation_df.isnull().values.any()

# dropping rows having null values
new_vaction_df=vacation_df.dropna(axis=0, how='any')

# Narrowing down the vacation site based on optimal weather conditions
hotel_df= new_vaction_df.loc[(new_vaction_df['Max Temperature'] < 72) & 
                                   (new_vaction_df['Max Temperature'] > 65) & 
                                   (new_vaction_df['Wind Speed'] < 5) & 
                                   (new_vaction_df['Cloudiness'] == 0) & (new_vaction_df['Humidity'] < 80)]
hotel_df

,Unnamed: 0,City,Country,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Date
100,100,Asunción,PY,-25.3007,-57.6359,71.04,64,0,4.61,1626230215
199,199,Kamenka,RU,51.3223,42.7678,69.03,65,0,4.21,1626230240
297,297,Ilovlya,RU,49.3033,43.9796,65.50,48,0,4.18,1626230266
395,395,Belyy Yar,RU,53.6039,91.3903,69.69,68,0,2.24,1626230292
435,435,Nicoreşti,RO,45.9322,27.3087,67.41,76,0,3.71,1626230302
543,543,Okhtyrka,UA,50.3104,34.8988,65.97,71,0,4.56,1626230331


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# use iterrows to iterate through pandas dataframe
base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    city = row['City']
    country = row['Country']
    params={
        'location':f'{lat},{lon}',
        'radius':5000,
        'type':'lodging',
        'keyword':'hotel',
        'key': g_key
    }
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(f"{base_url}", params=params).json()
    
    # extracting results
    results = response['results']
    
    try:
        # extracting hotel names from the json response
        hotelname = response['results'][0]['name']
        print(f"Closest hotel to the city {city} in the country {country} is {hotelname}.")
        
        # adding hotel names in the Hotel Name column
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Retrieving Results for Index 100: Asunción.
Closest hotel to the city Asunción in the country PY is Hotel Guaraní.
------------
Retrieving Results for Index 199: Kamenka.


C:\Users\Public\Anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Public\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Missing field/result... skipping.
------------
Retrieving Results for Index 297: Ilovlya.
Closest hotel to the city Ilovlya in the country RU is Dom U Dorogi.
------------
Retrieving Results for Index 395: Belyy Yar.
Missing field/result... skipping.
------------
Retrieving Results for Index 435: Nicoreşti.
Missing field/result... skipping.
------------
Retrieving Results for Index 543: Okhtyrka.
Closest hotel to the city Okhtyrka in the country UA is Kotedzh-Park "Slavna".
------------


In [7]:
# rearranging dataframe with required fields
available_hotel_df=hotel_df[["City", 'Country','Latitude','Longitude', 'Hotel Name']].reset_index(drop=True)
available_hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,Asunción,PY,-25.3007,-57.6359,Hotel Guaraní
1,Kamenka,RU,51.3223,42.7678,NaN
2,Ilovlya,RU,49.3033,43.9796,Dom U Dorogi
3,Belyy Yar,RU,53.6039,91.3903,NaN
4,Nicoreşti,RO,45.9322,27.3087,NaN
5,Okhtyrka,UA,50.3104,34.8988,"Kotedzh-Park ""Slavna"""


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in available_hotel_df.iterrows()]
locations = available_hotel_df[["Latitude", "Longitude"]]

In [9]:
# plotting gmap
fig = gmaps.figure(center=(18.0, 10.0), zoom_level=2)

# Adding heat layer gmap
heat_layer = gmaps.heatmap_layer(locations, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)
# Adding markers over heat layer
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))